In [13]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/project3/blink_data/blinkit_order_items.csv")
df # 결측치 확인

# 예: 값이 없다면 다른 테이블에서 가격 정보 가져오는 방식 고려

# pip install pymysql

,order_id,product_id,quantity,unit_price
0,1961864118,642612,3,517.03
1,1549769649,378676,1,881.42
2,9185164487,741341,2,923.84
3,9644738826,561860,1,874.78
4,5427684290,602241,2,976.55
...,...,...,...,...
4995,1669690997,925482,3,475.04
4996,8340761903,124290,2,39.65
4997,5936301790,491314,3,973.44
4998,5710579377,319388,2,657.80


In [14]:
#카테고리별 수

df = pd.read_csv("/content/drive/MyDrive/project3/blink_data/blinkit_order_items.csv")
products = pd.read_csv("/content/drive/MyDrive/project3/blink_data/blinkit_products.csv")

df_merged = pd.merge(df, products, on="product_id")
df_merged["total_revenue"] = df_merged["unit_price"] * df_merged["quantity"]

result = df_merged.groupby("category")["total_revenue"].sum().reset_index()
result = result.sort_values(by="total_revenue", ascending=False)

result.to_csv("/content/drive/MyDrive/project3/blink_data/top_categories.csv", index=False)

df_merged["category"].nunique()


11

In [15]:
#카테고리별 마진 , 총매출
# 병합된 데이터에 마진율 추가
df_merged = pd.merge(df, products, on="product_id")

# 매출액
df_merged["total_revenue"] = df_merged["unit_price"] * df_merged["quantity"]

# 마진액 = 매출 × (마진율 / 100)
df_merged["margin_amount"] = df_merged["total_revenue"] * (df_merged["margin_percentage"] / 100)

# 카테고리-상품 단위 집계
product_result = df_merged.groupby(["category", "product_name"]).agg({
    "total_revenue": "sum",
    "margin_amount": "sum"
}).reset_index()

# 저장
product_result.to_csv("/content/drive/MyDrive/project3/blink_data/product_by_category_margin.csv", index=False)


In [16]:
#브랜드별 마진, 총매출
df_merged = pd.merge(df, products, on="product_id")
df_merged["total_revenue"] = df_merged["unit_price"] * df_merged["quantity"]
df_merged["margin_amount"] = df_merged["total_revenue"] * (df_merged["margin_percentage"] / 100)

brand_product_result = df_merged.groupby(["brand", "product_name"]).agg({
    "total_revenue": "sum",
    "margin_amount": "sum"
}).reset_index()

brand_product_result.to_csv("/content/drive/MyDrive/project3/blink_data/brand_product_margin.csv", index=False)


In [17]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("/content/drive/MyDrive/project3/blink_data/blinkit_order_items.csv")
products = pd.read_csv("/content/drive/MyDrive/project3/blink_data/blinkit_products.csv")
orders = pd.read_csv("/content/drive/MyDrive/project3/blink_data/blinkit_orders.csv")
customers = pd.read_csv("/content/drive/MyDrive/project3/blink_data/blinkit_customers.csv")

# 병합
df_merged = df.merge(products, on="product_id") \
              .merge(orders, on="order_id") \
              .merge(customers, on="customer_id")

# 총 주문 금액(order_total) 추가
df_merged["order_total"] = df_merged["unit_price"] * df_merged["quantity"]


# 줄바꿈 제거 및 컬럼명 정리
df_clean = df_merged.replace({r'\n': ' ', r'\r': ' '}, regex=True)
df_clean.columns = df_clean.columns.str.strip().str.replace('[^A-Za-z0-9_]', '_', regex=True)

# 저장
import csv
df_clean.to_csv(
    "/content/drive/MyDrive/project3/blink_data/test_upload_final.csv",
    index=False,
    encoding="utf-8-sig",
    quoting=csv.QUOTE_MINIMAL  # 헤더 포함 최소한의 따옴표만
)




In [ ]:
from google.colab import files
uploaded = files.upload()


TypeError: 'NoneType' object is not subscriptable

In [ ]:
import pandas as pd
import csv

df = pd.read_csv("final_merged_data.csv")
df['total_revenue'] = df['unit_price'] * df['quantity']
df['margin_amount'] = df['total_revenue'] * (df['margin_percentage'] / 100)
# 고객별 총 수익 계산
customer_revenue = df.groupby("customer_id")["margin_amount"].sum().reset_index()

# 상위 20% 기준선 계산
threshold = customer_revenue["margin_amount"].quantile(0.8)

# 구분 필드 생성
customer_revenue["customer_group"] = customer_revenue["margin_amount"].apply(
    lambda x: "상위 20% 고객" if x >= threshold else "일반고객"
)

# 기존 df와 병합
df = df.merge(customer_revenue[["customer_id", "customer_group"]], on="customer_id", how="left")

df.to_csv(
    "/content/drive/MyDrive/project3/blink_data/plz171818.csv",
    index=False,
    encoding="utf-8-sig",
    quoting=csv.QUOTE_MINIMAL
)


df['brand'].nunique()

In [ ]:
df['category'].nunique()

In [ ]:
df['product_name'].nunique()

In [ ]:
df['product_id'].nunique()